# Introduction

In this notebook, I explore different approaches to evaluating a summarisation task. Techniques experimented below are common in the community, each with its own pros and cons. 

In [ ]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
sys.path.append('/mnt/d/Projects/papersurvey_tool/src/')

import os

In [ ]:
# Here I will be using the summariser created in the summarisation folder for the task
from summarisation.summariser import PaperSummariser
file_path = "../example_paper1.pdf"
autosum = PaperSummariser()
final_summary = autosum.summarise(file_path)

In [ ]:
ref_summary = " While deep learning has enabled tremendous progress on text and image datasets, its superiority on tabular data is not clear.\nWe contribute extensive benchmarks of standard and novel deep learning methods as well as tree-based models such as XGBoost and Random Forests, across a large number of datasets and hyperparameter combinations.\nWe deﬁne a standard set of 45 datasets from varied domains with clear characteristics of tabular data and a benchmarking methodology accounting for both ﬁtting models and ﬁnding good hyperparameters.\nResults show that treebased models remain state-of-the-art on medium-sized data (∼10K samples) even without accounting for their superior speed.\nTo understand this gap, we conduct an empirical investigation into the differing inductive biases of tree-based models and Neural Networks (NNs).\nThis leads to a series of challenges which should guide researchers aiming to build tabular-speciﬁc NNs: 1. be robust to uninformative features, 2. preserve the orientation of the data, and 3. be able to easily learn irregular functions.\nTo stimulate research on tabular architectures, we contribute a standard benchmark and raw data for baselines: every point of a 20 000 compute hours hyperparameter search for each learner."
eval_summary = "Tree-based models, such as XGBoost and Random Forests, consistently outperform deep learning models on medium-sized tabular datasets, even without considering their speed advantage. The inductive biases of tree-based models, such as their ability to handle irregular patterns and uninformative features, contribute to their superior performance. The lack of established benchmarks for tabular data and the challenges related to regularization techniques hinder the performance of deep learning models. Therefore, developing tabular-specific neural networks is necessary to address these challenges."

## Rouge score

ROUGE stands for Recall-Oriented Understandy for Gisting Evaluation and is often used for evaluting summarisation tasks. It measures the overlap of exact words between a generated text and a reference text. 

Cons: word-based, prone to miss the true essence of a good summary. May fail if a summary is articulated differently but still encapsulate the core information accurately. 

In [ ]:
from evaluation.evaluator import get_rouge_scores

rouge_scores = get_rouge_scores(ref_summary=ref_summary, eval_summary=eval_summary)
rouge_scores

## BERTScore

BERTScore leverages the contextual embedding from the BERT model to measure semantic similarities between two pieces of texts. 

Cons: may not fully grasp subtleties and high-level concepts that a human evalutor might understand.


In [ ]:
from evaluation.evaluator import get_bert_scores
bert_scores = get_bert_scores(ref_summary=ref_summary, eval_summary=eval_summary)
rouge_scores

## LLM as evaluator

Inspired by the G-eval framework, LLMs-as-evaluator has become a popular choice. LLM based evalutors can assess the quality of generated content based on the intruction prompt and the text without the need for ground truth references. 

Cons: costly, require careful design of the prompts, 

Here I will take the suggestions from openai cookbook and consider the following dimensions to score the quality of a summary. 
- Relevance: evaluates if the summary includes only important information and excludes redundancies
- Coherence: assesses the logical flow and organization of the summary
- Consistency: Checks if the summary aligns with the facts in the source document
- Fluency: Rates the grammer and readability of the summary

Cons: costly as for each score the evaluator will call the LLM; if the evaluation is done at section level summary then it is likely to hit the rate limit of the openai api

In [ ]:
from evaluation.evaluator import get_geval_scores
source_document = "\n".join(autosum.text_chunks)
geval_scores = get_geval_scores(document=source_document, eval_summary=eval_summary)

## References and useful resources

1. [How to evaluate a summarization task](https://cookbook.openai.com/examples/evaluation/how_to_eval_abstractive_summarization)
2. [G-EVAL: NLG Evaluation using GPT-4 with Better Human Alignment](https://arxiv.org/pdf/2303.16634.pdf)